<a href="https://colab.research.google.com/github/benmccloskey/Philadelpia_eagles_analysis/blob/main/Eagles_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [282]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


##Scraping for the Eagles Wins, Loses, and Ties from 1992 - 2021 Seasons

In [306]:
url = "https://www.jt-sw.com/football/pro/teams.nsf/histories/eagles"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

wins = []
loses = []
ties = []

for i in range(1992,2022):
  records = soup.find_all('a', href ="/football/pro/results.nsf/Teams/{}-phi".format(i))
  for rec in records:
    record = rec.text.strip().split('-')
    wins.append(record[0])
    loses.append(record[1])
    ties.append(record[2])

In [309]:
print('wins',wins)
print('loses', loses)
print('ties',ties)

wins ['11', '8', '7', '10', '10', '6', '3', '5', '11', '11', '12', '12', '13', '6', '10', '8', '9', '11', '10', '8', '4', '10', '10', '7', '7', '13', '9', '9', '4', '9']
loses ['5', '8', '9', '6', '6', '9', '13', '11', '5', '5', '4', '4', '3', '10', '6', '8', '6', '5', '6', '8', '12', '6', '6', '9', '9', '3', '7', '7', '11', '8']
ties ['0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0']


30

Superbowl Values: Went to Superbowl and Won Superbowl (2004 season - Went and lost. 2017 Season - went and won)

In [331]:
went_to_superbowl = []

for i in range(1992,2022):
  if i == 2004 or i == 2017:
    went_to_superbowl.append(1)
  else:
    went_to_superbowl.append(0) 

won_superbowl = [] 
for i in range(1992,2022):
  if i == 2017:
    won_superbowl.append(1)
  else:
    won_superbowl.append(0) 
went_to_superbowl

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0]

In [128]:
url = "https://www.philadelphiaeagles.com/team/stats/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
stats_list = soup.find_all("div", class_="nfl-o-team-h2h-stats__label")

words = []
for div in stats_list:
    word = div.text.strip()
    words.append(word)

print(words)


['TOTAL FIRST DOWNS TOTAL 1ST DOWNS', 'FIRST DOWNS 1ST DOWNS Rushing Passing By Penalty', 'THIRD DOWN CONVERSIONS 3RD DOWN CONVERSIONS', 'FOURTH DOWN CONVERSIONS 4TH DOWN CONVERSIONS', 'TOTAL OFFENSIVE YARDS', 'OFFENSE Plays Average Yards', 'TOTAL RUSHING YARDS', 'RUSHING Plays Average Yards', 'TOTAL PASSING YARDS', 'PASSING Comp Completions Att Attempts Int Interceptions Avg Average', 'SACKS', 'FIELD GOALS', 'TOUCHDOWNS', 'TOUCHDOWNS Rushing Passing Returns Defensive', 'TURNOVER RATIO']


In [338]:
col = ['TOTAL FIRST DOWNS TOTAL 1ST DOWNS', 'FIRST DOWNS 1ST DOWNS Rushing', 'FIRST DOWNS 1ST DOWNS Passing','FIRST DOWNS 1ST DOWNS By Penalty', 'THIRD DOWN CONVERSIONS', 'FOURTH DOWN CONVERSIONS', 'TOTAL OFFENSIVE YARDS', 'OFFENSE Plays',' OFFENSE Average Yards', 'TOTAL RUSHING YARDS', 'RUSHING Plays', 'Rushing Average Yards', 'TOTAL PASSING YARDS', 'PASSING Comp','PASSING Att' ,'PASSING Int', 'PASSING Avg', 'SACKS', 'FIELD GOALS', 'TOUCHDOWNS', 'TOUCHDOWNS Rushing','TOUCHDOWNS Passing','TOUCHDOWNS Returns','TOUCHDOWNS Defensive', 'TURNOVER RATIO']
#col = words

df = pd.DataFrame(columns= ['Year', 'Wins', 'Loses', 'Ties'] + col + ['Went to Superbowl', 'Won Superbowl'])

In [339]:
yr1 = 1992
yr2 = 2022
idx = 0
for yr in range(yr1,yr2):
  mapping = {}
  mapping['Year'] = yr
  mapping['Wins'] = wins[idx]
  mapping['Loses'] = loses[idx]
  mapping['Ties'] = ties [idx]
  url = "https://www.philadelphiaeagles.com/team/stats/{}/REG".format(yr)
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")
  stats_list = soup.find("ul", class_="nfl-o-team-h2h-stats__list")

  temp_df = pd.DataFrame()
  vals = []
  for i,li in enumerate(stats_list.find_all("li")):
      next_col = li.find_all("div", class_="nfl-o-team-h2h-stats__value")[0]
      value = next_col.text.strip()
      vals.append(value)
  temp_val = []
  for v in vals:
    temp = []
    if '\n' in v:
      temp = v.split('\n')
      for w in temp:
        temp_val.append(w)
    else:
      temp_val.append(v)
  for i,c in enumerate(col):
    mapping[c] = temp_val[i]
  
  mapping['Went to Superbowl'] = went_to_superbowl[idx]
  mapping['Won Superbowl'] = won_superbowl[idx]
  idx += 1
  temp_df = pd.DataFrame.from_dict(mapping,orient='index').T
  df = df.append(temp_df, ignore_index=True)

In [342]:
df

,Year,Wins,Loses,Ties,TOTAL FIRST DOWNS TOTAL 1ST DOWNS,FIRST DOWNS 1ST DOWNS Rushing,FIRST DOWNS 1ST DOWNS Passing,FIRST DOWNS 1ST DOWNS By Penalty,THIRD DOWN CONVERSIONS,FOURTH DOWN CONVERSIONS,...,SACKS,FIELD GOALS,TOUCHDOWNS,TOUCHDOWNS Rushing,TOUCHDOWNS Passing,TOUCHDOWNS Returns,TOUCHDOWNS Defensive,TURNOVER RATIO,Went to Superbowl,Won Superbowl
0,1992,11,5,0,3731,1769,1756,206,1029 / 2658,89 / 177,...,64,16 / 25,43,19,20,1,3,9,0,0
1,1993,8,8,0,3828,1327,2315,186,1165 / 2817,89 / 198,...,42,16 / 23,34,7,23,0,5,1,0,0
2,1994,7,9,0,3694,1308,2114,272,1282 / 3057,74 / 210,...,48,21 / 25,35,14,18,1,2,9,0,0
3,1995,10,6,0,3672,1597,1835,240,1131 / 2947,125 / 212,...,46,22 / 30,32,19,11,1,4,2,0,0
4,1996,10,6,0,4314,1453,2643,218,1120 / 2959,81 / 202,...,39,25 / 29,38,16,19,2,4,-1,0,0
5,1997,6,9,1,4328,1406,2690,232,1199 / 3209,183 / 290,...,64,22 / 31,35,11,22,1,2,-6,0,0
6,1998,3,13,0,3394,1137,1841,416,1053 / 3155,91 / 298,...,56,14 / 21,17,10,7,0,0,-9,0,0
7,1999,5,11,0,2905,1011,1637,257,982 / 3174,79 / 120,...,49,21 / 31,29,5,18,1,5,7,0,0
8,2000,11,5,0,295,88,182,25,90 / 224,7 / 11,...,45,29 / 33,38,13,21,2,2,2,0,0
9,2001,11,5,0,256,90,146,20,68 / 220,9 / 13,...,40,26 / 31,36,6,27,1,4,9,0,0


In [343]:

path = '/content/drive/MyDrive/eagles_Stats.csv'
with open(path, 'w', encoding = 'utf-8-sig') as f:
  df.to_csv(f)
